In [287]:
import time
import datetime
#----------
import numpy as np
from datetime import datetime
import pandas as pd
import json
import os
import threading
import csv
#----------
import configparser

In [299]:
###############################################################################################
#----------------------------------------Binance API------------------------------------------#
###############################################################################################
import requests 
import hmac
import hashlib

class binanceAPI():
    def __init__(self,host,key,secret):
        self._api_host = host
        self._api_key = key
        self._api_secret  = secret
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self.header = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-MBX-APIKEY': self._api_key,
        }
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(_api_secret, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs ):
        res = requests.get(self._api_host + url,**kwargs)
        return json.loads(res.text)
    
    def _post(self,url,data ):
        signature = self._sign(data)
        data['sig'] = signature
        res = requests.post(self.API_HOST + url, headers=self.header, data=self._json_encode(data))
        return json.loads(res.text)
    
    def server_time(self):
        res = self._get(f'/fapi/v1/time')
        return res['serverTime']
    
    def get_ticker(self,market_name):
        res = self._get(f'/fapi/v1/depth?symbol={market_name}&limit=5')
        return {'bids':res['bids'][0][0],
                'bidv':res['bids'][0][1],
                'asks':res['bids'][0][0],
                'askv':res['bids'][0][1]}

    def timeframe(self,tf):
        if(tf==60):
            return '1m'
        elif(tf==300):
            return '5m'
        elif(tf==900):
            return '15m'
        elif(tf==3600):
            return '1h'
        elif(tf==11400):
            return '1h'
        elif(tf==86400):
            return '1d'
        
        
    def exchangeInfo(self):
        return self._get(f'/fapi/v1/exchangeInfo')
    
    def _listToDict_HistoricalPriceFN(self,price_list):
        price_dict={'time':[],
                 'open':[],
                 'high':[],
                 'low':[],
                 'close':[],
                 'volume':[]
                }
        for i in price_list:
            price_dict["time"].append(self.timestampToDatetime(int(i[0])/1000))
            price_dict["open"].append(float(i[1]))
            price_dict["high"].append(float(i[2]))
            price_dict["low"].append(float(i[3]))
            price_dict["close"].append(float(i[4]))
            price_dict["volume"].append(float(i[5]))
        return price_dict
    
    def timestampToDatetime(self,tm):
        time_tuple = time.gmtime(tm)
        return time.strftime("%Y-%m-%d %H:%M:%S", time_tuple)
    
    def datetimeToTimestamp(self,tm):
        time_tuple = time.strptime(tm, "%Y-%m-%d %H:%M:%S")
        return int(time.mktime(time_tuple))

    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        try:
            res = self._get(f'/fapi/v1/klines?symbol={market_name}&interval={self.timeframe(resolution)}&startTime={int(start_time*1000)}&endTime={int(end_time*1000)}&limit={limit}')
            return self._listToDict_HistoricalPriceFN(res)
        except:
            return res
    


In [300]:
################################# Test ###############################

In [301]:
config = configparser.ConfigParser()
config.read('config.ini') 

#API
api = binanceAPI(
    config['API']['host'],
    config['API']['api_key'].encode(),
    config['API']['secret_key']
)

market_name = config['SYSTEM']['market_name'].upper()
timeframe = config['SYSTEM']['timeframe'].split(',')
timeframe = [int(i) for i in timeframe]
nbar =      int(config['SYSTEM']['nbar'])
start_time= config['SYSTEM']['start_time']
end_time =  config['SYSTEM']['end_time']


In [302]:
def iOBV(close_list,vol_list):
    obv_list = list([])
    for i in range(len(vol_list)):
        if(i>0):
            if(close_list[i]>close_list[i-1]):
                obv=obv+vol_list[i]
            elif(close_list[i]==close_list[i-1]):
                obv=obv
            elif(close_list[i]<close_list[i-1]):
                obv=obv-vol_list[i]
        else:
            obv= 0
        obv_list.append(round(obv,2))
    return obv_list

In [303]:
def iRSI(data,n):
    rsi = list([])
    for i in range(len(data)):
        if i > n+1:
            sample = data[i-n-1:i]
            avgUP=0
            avgDown=0
            for s in range(len(sample)):
                if(sample[s]>sample[s-1] and s>0):  
                    avgUP += 1
                else:
                    avgDown += 1
            if(avgUP==0):
                rs =  0
            elif(avgDown==0):
                rs = 1
            else:
                rs = avgUP/avgDown
            rsi.append(round(100 - 100/(1+rs) ,2) )
        else:
            rsi.append(0)
    return rsi

In [304]:
def iEMA(data,n,*smooth):
    if(len(smooth)==0):
        smooth=2
    ema=list([])
    for i in range(len(data)):
        if i > n:
            ema.append(data[i]*(smooth/1+n) + ema[i-1]*(1-(smooth/1+n)))
        else:
            ema.append(0)
    return ema      

In [305]:
def calculate(sym,data):
    vol_4h = data[0]['volume'].tolist()
    close_4h = data[0]['close'].tolist()
    obv_4h = iOBV(close_4h,vol_4h)
    rsi_obv_4h = iRSI(obv_4h,14)
    ema_obv_4h_13 = iEMA(obv_4h,13)
    ema_close_4h_13 = iEMA(close_4h,13)
    data[0]['OBV']=obv_4h
    data[0]['rsi_obv_14']=rsi_obv_4h
    data[0]['ema_obv_13']=ema_obv_4h_13
    data[0]['ema_close_13']=ema_close_4h_13
    data[0]['symbol']=f'{sym}_4h'

    vol_1d = data[1]['volume'].tolist()
    close_1d = data[1]['close'].tolist()
    obv_1d = iOBV(vol_1d,vol_1d)
    ema_obv_1d_55 = iEMA(obv_1d,55)
    ema_close_1d_27 = iEMA(close_1d,27)
    data[1]['OBV']=obv_1d
    data[1]['ema_obv_55']=ema_obv_1d_55
    data[1]['ema_close_27']=ema_close_1d_27
    data[0]['symbol']=f'{sym}_1d'
    conditon = all([obv_4h[-1:][0] >= ema_obv_4h_13[-1:][0]
            , obv_1d[-1:][0] >= ema_obv_1d_55[-1:][0]
            , close_1d[-1:][0] >= ema_close_1d_27[-1:][0]
            , close_4h[-1:][0] >= ema_close_4h_13[-1:][0]
            , float(rsi_obv_4h[-1:][0]) > 60])
    if(conditon):
        tm=data[0]['time']
        price=data[0]['close']
        print(f'#####{sym} {tm} {price}')
    return data

In [313]:
exchange = api.exchangeInfo()

In [334]:
market_name = 'all'
dataList=list([])
if(market_name == 'all'):
    exchanges = api.exchangeInfo()
    for exchange in exchanges['symbols']:
        data = list([])
        sym = exchange['symbol']
        print(f'---{sym}')
        for tf in timeframe:
            data.append(pd.DataFrame.from_dict(api.historicalPrice(sym,tf,nbar,api.datetimeToTimestamp(start_time),api.datetimeToTimestamp(end_time)))) 
        data = calculate(market_name,data)
        data.append(data)
else:
    data = list([])
    for tf in timeframe:
        data.append(pd.DataFrame.from_dict(api.historicalPrice(market_name,tf,nbar,api.datetimeToTimestamp(start_time),api.datetimeToTimestamp(end_time)))) 
    data = calculate(market_name,data)
    
print('---complete---')



---BTCUSDT
---ETHUSDT
---BCHUSDT
---XRPUSDT
---EOSUSDT
---LTCUSDT
---TRXUSDT
---ETCUSDT
---LINKUSDT
---XLMUSDT
---ADAUSDT
---XMRUSDT
---DASHUSDT
---ZECUSDT
---XTZUSDT
---BNBUSDT
---ATOMUSDT
---ONTUSDT
---IOTAUSDT
---BATUSDT
---VETUSDT
---NEOUSDT
---QTUMUSDT
---IOSTUSDT
---THETAUSDT
---ALGOUSDT
---ZILUSDT
---KNCUSDT
---ZRXUSDT
---COMPUSDT
---OMGUSDT
---DOGEUSDT
---SXPUSDT
---KAVAUSDT
---BANDUSDT
---RLCUSDT
---WAVESUSDT
---MKRUSDT
---SNXUSDT
---DOTUSDT
---DEFIUSDT
---YFIUSDT
---BALUSDT
---CRVUSDT
---TRBUSDT
---YFIIUSDT
---RUNEUSDT
---SUSHIUSDT
---SRMUSDT
---BZRXUSDT
---EGLDUSDT
---SOLUSDT
---ICXUSDT
---STORJUSDT
---BLZUSDT
---UNIUSDT
---AVAXUSDT
---FTMUSDT
---HNTUSDT
---ENJUSDT
---FLMUSDT
---TOMOUSDT
---RENUSDT
---KSMUSDT
---NEARUSDT
---AAVEUSDT
---FILUSDT
---RSRUSDT
---LRCUSDT
---MATICUSDT
---OCEANUSDT
---CVCUSDT
---BELUSDT
---CTKUSDT
---AXSUSDT
---ALPHAUSDT
---ZENUSDT
---SKLUSDT
---GRTUSDT
---1INCHUSDT
---BTCBUSD
---AKROUSDT
---CHZUSDT
---SANDUSDT
---ANKRUSDT
---LUNAUSDT
---BTSUSDT
---

IndexError: list index out of range